# Notebook for plotting calibration curves for all calibration shots

In [1]:
%matplotlib tk
import numpy as np
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt
import sys
import glob
import os
import re
import gc
sys.path.append('/home/jleland/Coding/Projects/flopter')
import flopter.core.ivdata as iv
import flopter.core.lputils as lp
import flopter.magnum.database as ut
import flopter.core.fitters as fts
import flopter.magnum.magoptoffline as mg

In [2]:
line_defaults = {
    'color': 'grey',
    'linewidth': .9,
    'linestyle': '--'
}

In [3]:
os.chdir('/home/jleland/Data/')
adcfile_metadata = pd.read_csv('magnum2_probe-configurations.csv').set_index('adc_index')

In [4]:
indices = [0,2,33,35,36,81,82,120,202,212,215,217,218,228,231,233,257,
           260,272,334,352,353,355,368,387,438,451,473,487,489,502]

In [5]:
calibration_df = adcfile_metadata.loc[indices]
calibration_df.at[2, 'adc_4_probe'] = 'resistor'
calibration_df.at[2, 'adc_5_probe'] = 'empty'
calibration_df.loc[2]

shot_number                                         -
adc_time                          2019-05-28 11:21:35
adc_timestamp                     6696021067838019493
adc_4_coax                                          1
adc_4_probe                                  resistor
adc_4_shunt_resistance                             10
adc_5_coax                                          2
adc_5_probe                                     empty
adc_5_shunt_resistance                             10
adc_calibration_index                               0
adc_calibration_fl                              False
shot_fl                                         False
notes                     Test through 99ohm resistor
Name: 2, dtype: object

In [6]:
print(f'{len(calibration_df.index.values)}, {len(indices)}')

31, 31


In [7]:
folders = [
    '2019-05-28_Leland/', 
    '2019-05-29_Leland/', 
    '2019-06-03_Leland/',
    '2019-06-04_Leland/', 
    '2019-06-05_Leland/',
    '2019-06-06_Leland/',
    '2019-06-07_Leland/'
]
mg.Magoptoffline._FOLDER_STRUCTURE = '/Data/external/magnum/'

files = []
file_folders = []
for folder1 in folders:
    os.chdir(mg.Magoptoffline.get_data_path() + folder1)
    files.extend(glob.glob('*.adc'))
    file_folders.extend([folder1] * len(glob.glob('*.adc')))
files.sort()


In [8]:
# for i, f in enumerate(files):
#     print(f'{i:.3g}, {file_folders[i]}, {f}')

In [8]:
for i , index in enumerate(indices):
    row = calibration_df.loc[index]
    print([(fi, f) for fi, f in enumerate(files) if str(row['adc_timestamp']) in f])

[(0, '2019-05-28 10h 30m 08s TT_06695992307612752513.adc')]
[(2, '2019-05-28 12h 21m 44s TT_06696021067838019493.adc')]
[(33, '2019-05-29 10h 53m 12s TT_06696369343194399872.adc')]
[(35, '2019-05-29 10h 59m 05s TT_06696370858617159264.adc')]
[(36, '2019-05-29 11h 09m 34s TT_06696373559952702292.adc')]
[(81, '2019-05-29 15h 17m 30s TT_06696437439036956970.adc')]
[(82, '2019-05-29 15h 31m 17s TT_06696440987104719837.adc')]
[(120, '2019-06-03 12h 13m 57s TT_06698245570768084051.adc')]
[(202, '2019-06-03 17h 39m 20s TT_06698329408184736844.adc')]
[(212, '2019-06-03 18h 13m 06s TT_06698338110793349459.adc')]
[(215, '2019-06-04 10h 24m 17s TT_06698588393799761719.adc')]
[(217, '2019-06-04 11h 46m 06s TT_06698609479441234691.adc')]
[(218, '2019-06-04 12h 03m 56s TT_06698614073190320148.adc')]
[(228, '2019-06-04 14h 02m 53s TT_06698644726189167348.adc')]
[(231, '2019-06-04 14h 12m 15s TT_06698647139408299525.adc')]
[(233, '2019-06-04 14h 24m 17s TT_06698650240585248344.adc')]
[(257, '2019-06-0

In [9]:
row = calibration_df.loc[0]
row

shot_number                                                            -
adc_time                                             2019-05-28 09:29:58
adc_timestamp                                        6695992307612752513
adc_4_coax                                                             1
adc_4_probe                                                            S
adc_4_shunt_resistance                                                10
adc_5_coax                                                             2
adc_5_probe                                                            L
adc_5_shunt_resistance                                                10
adc_calibration_index                                                  -
adc_calibration_fl                                                  True
shot_fl                                                            False
notes                     Test of adc on probes - shows voltage waveform
Name: 0, dtype: object

In [10]:
def calibration_analysis(index):
    row = calibration_df.loc[index]
    files_index, adc_file = [(fi, f) for fi, f in enumerate(files) if str(row['adc_timestamp']) in f][0]
    folder = file_folders[files_index]
    shunt_resistor = float(row['adc_4_shunt_resistance'])
    
    dsr = 10
    cabling_resistance = 1.2 + 0.7
    
#     print(folder, adc_file, shunt_resistor, cabling_resistance)
    
    magopter = mg.Magoptoffline(folder, adc_file, shunt_resistor=shunt_resistor, 
                                cabling_resistance=cabling_resistance)
    
    freq = len(magopter.m_data.time) / magopter.adc_duration
    print(magopter.adc_duration, len(magopter.m_data.time), freq)
    print(int(freq / 10000))
    dsr = max(1, freq / 10000)
    
    magopter._VOLTAGE_CHANNEL = 3
    magopter._PROBE_CHANNEL_3 = 4
    magopter._PROBE_CHANNEL_4 = 5
    magopter.prepare(down_sampling_rate=dsr, roi_b_plasma=True, filter_arcs_fl=False, crit_freq=None, 
                     crit_ampl=None)
    
    probes = [row['adc_4_probe'], row['adc_5_probe']]
    print(f'{index}, {probes} \n'
          f'\tchannel_0 mean: {np.mean(magopter.current[0])}, \n'
          f'\tchannel_1 mean: {np.mean(magopter.current[1])}')
    
    ds_full = magopter.to_xarray(probes)
    
    fig, ax = plt.subplots(2, sharex=True)
    ds_full.mean(['sweep']).set_coords('voltage').sel(probe=probes[0])['current'].plot.line(x='voltage', hue='direction', ax=ax[0])
    ds_full.mean(['sweep']).set_coords('voltage').sel(probe=probes[1])['current'].plot.line(x='voltage', hue='direction', ax=ax[1])
    plt.savefig(f'callibiv_{index}.png')
                
    return np.mean(magopter.current[0]), np.mean(magopter.current[1])

In [12]:
calibration_analysis(334)

Running an offline magopter object, operating in offline mode.
22.0 22000000 1000000.0
100


/home/jleland/Coding/Environments/python3.6/lib/python3.6/site-packages/scipy/signal/_arraytools.py:45: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  b = a[a_slice]


334, ['R', 'B'] 
	channel_0 mean: 0.0005021520016455942, 
	channel_1 mean: 0.0010001890057522182


(0.0005021520016455942, 0.0010001890057522182)

In [13]:
import concurrent.futures as cf

os.chdir('/home/jleland/Data/external/magnum/calibration/')

offsets = []
for index in indices:
    print(f'Analysing index {index}')
    offsets.append(calibration_analysis(index))
plt.show()

print(offsets)

Analysing index 0
Running an offline magopter object, operating in offline mode.
10.0 100000 10000.0
1


/home/jleland/Coding/Environments/python3.6/lib/python3.6/site-packages/scipy/signal/_arraytools.py:45: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  b = a[a_slice]


0, ['S', 'L'] 
	channel_0 mean: 0.0016296950759616417, 
	channel_1 mean: 0.001137472269416819
Analysing index 2
Running an offline magopter object, operating in offline mode.
10.0 1000000 100000.0
10


/home/jleland/Coding/Environments/python3.6/lib/python3.6/site-packages/scipy/signal/_arraytools.py:45: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  b = a[a_slice]


2, ['resistor', 'empty'] 
	channel_0 mean: -0.37783118198469995, 
	channel_1 mean: 0.0011659376454164726
Analysing index 33
Running an offline magopter object, operating in offline mode.
8.0 800000 100000.0
10


...

Analysing index 502
Running an offline magopter object, operating in offline mode.
10.0 10000000 1000000.0
100


/home/jleland/Coding/Environments/python3.6/lib/python3.6/site-packages/scipy/signal/_arraytools.py:45: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  b = a[a_slice]


502, ['S', 'L'] 
	channel_0 mean: 0.00039105268828790337, 
	channel_1 mean: 0.0004347930351596716


/home/jleland/Coding/Environments/python3.6/lib/python3.6/site-packages/matplotlib/pyplot.py:537: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


[(0.0016296950759616417, 0.001137472269416819), (-0.37783118198469995, 0.0011659376454164726), (0.00156763279818985, 0.0011302065860811412), (0.0015485077039112162, 0.0010911186381984048), (0.001550522572998599, 0.0011360178341467638), (0.00168363322917789, 0.0012032277497015017), (0.0017011672592752213, 0.0012248494883143773), (0.0020370218726430346, 0.0008869025073858039), (0.0020081987186394037, 0.0008630182029166953), (0.0019410375208241816, -0.06418549377294926), (-0.04695946557483031, -0.03963221487549643), (8.008619760801938e-05, -0.0002927016351575121), (0.00022207089753259669, -0.00023584014198058825), (0.00022805301152893328, -6.72987977101326e-05), (0.0002601702402758356, -6.698713300940647e-05), (0.0002786165283913404, 3.4078803553840907e-05), (0.0004409223144057787, 3.144696830326464e-05), (0.0004691304816291384, -0.00012287034532016213), (-1.1464281866149416e-06, 2.2024305517956056e-06), (0.0005021520016455942, 0.0010001890057522182), (-0.00016191262914676076, 5.774608224

In [18]:
for i, (ch0, ch1) in enumerate(offsets):
    print(i, indices[i],'\t', f'{ch0:.5g}, {ch1:.5g}')

0 0 	 0.0016297, 0.0011375
1 2 	 -0.37783, 0.0011659
2 33 	 0.0015676, 0.0011302
3 35 	 0.0015485, 0.0010911
4 36 	 0.0015505, 0.001136
5 81 	 0.0016836, 0.0012032
6 82 	 0.0017012, 0.0012248
7 120 	 0.002037, 0.0008869
8 202 	 0.0020082, 0.00086302
9 212 	 0.001941, -0.064185
10 215 	 -0.046959, -0.039632
11 217 	 8.0086e-05, -0.0002927
12 218 	 0.00022207, -0.00023584
13 228 	 0.00022805, -6.7299e-05
14 231 	 0.00026017, -6.6987e-05
15 233 	 0.00027862, 3.4079e-05
16 257 	 0.00044092, 3.1447e-05
17 260 	 0.00046913, -0.00012287
18 272 	 -1.1464e-06, 2.2024e-06
19 334 	 0.00050215, 0.0010002
20 352 	 -0.00016191, 5.7746e-05
21 353 	 5.0969e-05, -2.5068e-05
22 355 	 5.1882e-05, 9.8178e-05
23 368 	 7.8569e-05, 2.8902e-05
24 387 	 -8.4732e-06, 0.00012034
25 438 	 0.00035356, 0.00024028
26 451 	 -0.00010564, 9.9165e-05
27 473 	 0.00042796, 0.00040487
28 487 	 0.00052554, 0.00032713
29 489 	 0.00040143, 0.00049496
30 502 	 0.00039105, 0.00043479


## Closer look at some of the more interesting looking ones
Some of the IV characteristics either weren't flat or didn't seem to have a voltage bias, will now take a closer look at them, in particular figures 10, 11, 20, and 21, which correspond to indices 9, 10, 19 and 20 respectively.

In [11]:
indices_oi = [9, 10, 19, 20]

In [12]:
index = indices[indices_oi[3]]

row = calibration_df.loc[index]
files_index, adc_file = [(fi, f) for fi, f in enumerate(files) if str(row['adc_timestamp']) in f][0]
folder = file_folders[files_index]
shunt_resistor = float(row['adc_4_shunt_resistance'])

dsr = 100
# cabling_resistance = 1.2 + 0.7
cabling_resistance = 1.7 + 6.09 + 1.25 + 1.0

#     print(folder, adc_file, shunt_resistor, cabling_resistance)

magopter = mg.Magoptoffline(folder, adc_file, shunt_resistor=shunt_resistor, 
                            cabling_resistance=cabling_resistance)

freq = len(magopter.m_data.time) / magopter.adc_duration
print(magopter.adc_duration, len(magopter.m_data.time), freq)
print(int(freq / 10000))
# dsr = max(1, freq / 10000)

magopter._VOLTAGE_CHANNEL = 3
magopter._PROBE_CHANNEL_3 = 4
magopter._PROBE_CHANNEL_4 = 5
magopter.prepare(down_sampling_rate=dsr, roi_b_plasma=True, filter_arcs_fl=False, crit_freq=None, 
                 crit_ampl=None)

probes = [row['adc_4_probe'], row['adc_5_probe']]
print(f'{index}, {probes} \n'
      f'\tchannel_0 mean: {np.mean(magopter.current[0])}, \n'
      f'\tchannel_1 mean: {np.mean(magopter.current[1])}')

Running an offline magopter object, operating in offline mode.
18.0 18000000 1000000.0
100


/home/jleland/Coding/Environments/python3.6/lib/python3.6/site-packages/scipy/signal/_arraytools.py:45: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  b = a[a_slice]
/home/jleland/Coding/Environments/python3.6/lib/python3.6/site-packages/numpy/core/_methods.py:140: RuntimeWarning: Degrees of freedom <= 0 for slice
  keepdims=keepdims)
/home/jleland/Coding/Environments/python3.6/lib/python3.6/site-packages/numpy/core/_methods.py:110: RuntimeWarning: invalid value encountered in true_divide
  arrmean, rcount, out=arrmean, casting='unsafe', subok=False)
/home/jleland/Coding/Environments/python3.6/lib/python3.6/site-packages/numpy/core/_methods.py:132: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


352, ['S', 'L'] 
	channel_0 mean: -0.00016191262914676076, 
	channel_1 mean: 5.7746082247253955e-05


In [13]:
print(f'{index}, {probes} \n'
      f'\tchannel_0 mean: {np.mean(magopter.voltage[0])}, \n'
      f'\tchannel_1 mean: {np.mean(magopter.voltage[1])}')

print(np.mean(magopter.voltage))
print(np.mean(magopter.m_data.data[3]))

352, ['S', 'L'] 
	channel_0 mean: 0.00177866974244551, 
	channel_1 mean: -0.0006463624313444191
0.0005661536555505455
0.11611091154316663


In [21]:
fig, ax = plt.subplots(2, sharex=True)

ax[0].plot(magopter.raw_time, magopter.voltage[0])
ax[0].plot(magopter.raw_time, magopter.voltage[1])
ax[0].set_ylabel(r'$V_P$ (V)')
ax[1].plot(magopter.raw_time, magopter.current[0])
ax[1].plot(magopter.raw_time, magopter.current[1])
ax[1].set_ylabel(r'$I$ (A)')
plt.show()

In [31]:
fig, ax = plt.subplots(2, sharex=True)

ax[0].plot(magopter.raw_time, magopter.m_data.data[3])
ax[0].set_ylabel(r'$V_P$ (V)')
ax[1].plot(magopter.raw_time, magopter.voltage[0])
ax[1].plot(magopter.raw_time, magopter.voltage[1])
ax[1].set_ylabel(r'$I$ (A)')
plt.show()

In [23]:
ds_full = magopter.to_xarray(probes)
ds_full

<xarray.Dataset>
Dimensions:     (direction: 2, probe: 2, sweep: 35982, time: 2)
Coordinates:
  * time        (time) float64 0.0 0.0001
  * direction   (direction) <U4 'up' 'down'
  * probe       (probe) <U1 'S' 'L'
Dimensions without coordinates: sweep
Data variables:
    voltage     (probe, direction, sweep, time) float64 0.165 ... -0.06994
    current     (probe, direction, sweep, time) float64 -0.005016 ... 0.0005381
    shot_time   (probe, direction, sweep, time) float64 1.008 1.008 ... 18.99
    start_time  (probe, direction, sweep) float64 1.008 1.009 ... 18.99 18.99

In [24]:
fig, ax = plt.subplots(2, sharex=True)
ds_full.mean(['sweep']).set_coords('voltage').sel(probe=probes[0])['current'].plot.line(x='voltage', hue='direction', ax=ax[0])
ds_full.mean(['sweep']).set_coords('voltage').sel(probe=probes[1])['current'].plot.line(x='voltage', hue='direction', ax=ax[1])
plt.show()

print(np.mean(magopter.current[0]), np.mean(magopter.current[1]))

-0.00016191262914676076 5.7746082247253955e-05


### Plotting and Fitting of Weird IV Signal on B in 334 

In [100]:
ds_weird_iv = ds_full.mean(['sweep', 'direction']).set_coords('voltage').sel(probe=probes[1])

In [105]:
ds_weird_iv['current'].plot.line(x='voltage')

In [112]:
iv_data = iv.IVData(ds_weird_iv['voltage'].values,
                    -ds_weird_iv['current'].values + 0.0005036455711081326,
                    ds_weird_iv['shot_time'].values,
                    estimate_error_fl=True,)

fig, ax = plt.subplots()
ax.plot(iv_data['V'], iv_data['I'])
fig.show()

dict_keys(['I', 'V', 't', 'sigma'])


In [114]:
fit_data = iv_data.multi_fit(sat_region=-60)
fit_data.plot()

### Dataset plotting to figure out what is happening in 334 with the Voltage drift

In [67]:
fig, ax = plt.subplots(2)
ds_full.min(['time']).sel(probe='B')['voltage'].plot.line(hue='direction', ax=ax[0])
ds_full.max(['time']).sel(probe='B')['voltage'].plot.line(hue='direction', ax=ax[1])

In [61]:
fig, ax = plt.subplots(2)
ds_full.mean(['sweep']).sel(probe='R')['current'].plot.line(hue='direction', ax=ax[0])
ds_full.mean(['sweep']).sel(probe='B')['current'].plot.line(hue='direction', ax=ax[1])
plt.show()

In [63]:
fig, ax = plt.subplots(2)
ds_full.mean(['sweep']).sel(probe='R')['voltage'].plot.line(hue='direction', ax=ax[0])
ds_full.mean(['sweep']).sel(probe='B')['voltage'].plot.line(hue='direction', ax=ax[1])
plt.show()

In [68]:
fig, ax = plt.subplots(2)
ds_full.sel(probe='B', direction='up')['voltage'].plot.line(hue='sweep', ax=ax[0])
ds_full.sel(probe='B', direction='down')['voltage'].plot.line(hue='sweep', ax=ax[1])
plt.show()

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/home/jleland/Coding/Environments/python3.6/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 2961, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-68-71a88861f138>", line 2, in <module>
    ds_full.sel(probe='B', direction='up')['voltage'].plot.line(hue='sweep', ax=ax[0])
  File "/home/jleland/Coding/Environments/python3.6/lib/python3.6/site-packages/xarray/plot/plot.py", line 431, in line
    return line(self._da, *args, **kwargs)
  File "/home/jleland/Coding/Environments/python3.6/lib/python3.6/site-packages/xarray/plot/plot.py", line 301, in line
    title=huelabel)
  File "/home/jleland/Coding/Environments/python3.6/lib/python3.6/site-packages/matplotlib/axes/_axes.py", line 557, in legend
    self.legend_ = mlegend.Legend(self, handles, labels, **kwargs)
  File "/home/jleland/Coding/Environments/python3.6/lib/python3.6/site-packages/matplotlib/legend.py", line 700, in __init__
  

KeyboardInterrupt: 

In [87]:
fig, ax = plt.subplots()
ax.plot(magopter.raw_time, magopter.voltage[1])

# ds_full.sel(probe='B').isel(time=0).sel(direction=['up'])['start_time'].values[0]
for up_sttime in ds_full.sel(probe='B').isel(time=0).sel(direction=['up'])['start_time'].values[0]:
#     print(up_sttime)
    ax.axvline(x=up_sttime, **line_defaults)  
    
for dn_sttime in ds_full.sel(probe='B').isel(time=0).sel(direction=['down'])['start_time'].values[0]:
    ax.axvline(x=dn_sttime, **line_defaults)    


1.0050000002272728
1.0150000006818183
1.0250000011363636
1.035000001590909
1.0450000020454546
1.0550000025000001
1.0650000029545457
1.075000003409091
1.0850000038636365
1.095000004318182
1.1050000047727275
1.115000005227273
1.1250000056818183
1.1350000061363639
1.1450000065909094
1.155000007045455
1.1650000075000004
1.1750000079545457
1.1850000084090913
1.1950000088636368
1.2050000093181823
1.2150000097727278
1.2250000102272731
1.2350000106818186
1.2450000111363642
1.2550000115909097
1.265000012045455
1.2750000125000005
1.285000012954546
1.2950000134090915
1.305000013863637
1.3150000143181826
1.3250000147727279
1.3350000152272734
1.345000015681819
1.3550000161363644
1.3650000165909097
1.3750000170454553
1.3850000175000008
1.3950000179545463
1.4050000184090918
1.4150000188636374
1.4250000193181827
1.4350000197727282
1.4450000202272737
1.455000020681819
1.4650000211363645
1.47500002159091
1.4850000220454556
1.495000022500001
1.5050000229545466
1.5150000234090921
1.5250000238636374
1.5350

## Calculations for correcting Voltage and Current Offsets for ADC

In [40]:
print(ds_full.mean(['sweep', 'time', 'direction'])['voltage'].values)
print(ds_full.mean(['sweep', 'time', 'direction', 'probe'])['voltage'].values)

print(f'{index}, {probes} \n'
      f'\tchannel_0 mean: {np.mean(magopter.voltage[0])}, \n'
      f'\tchannel_1 mean: {np.mean(magopter.voltage[1])}')

print(np.mean(magopter.voltage))

[ 0.00147042 -0.00102681]
0.00022180444841530685
352, ['S', 'L'] 
	channel_0 mean: 0.0016774754683491295, 
	channel_1 mean: -0.0007475567054407988
0.0004649593814541654


In [29]:
(-0.26879880572590203 - 0.12) / 100

-0.00388798805725902

In [19]:
0.12 - 0.00388798805725902

0.11611201194274097

In [17]:
print(0.11611201194274097 - (0.0004649593814541654 / 100))
print(0.11611091154316663)

0.11610736234892642
0.11611091154316663


In [20]:
0.05 - 0.00032542902593960237

0.0496745709740604